# Catalyst Cooperative Jupyter Notebook Template
This notebook lays out a standard format and some best practices for creating interactive / exploratory notebooks which can be relatively easily shared between different PUDL users, and turned into reusable Python modules for integration into our underlying Python packages.

## Begin with a narrative outline
Each notebook should start with a brief explanation (in Markdown) explaining the purpose of the analysis, and outlining the different stages / steps which are taken to accomplish the analysis.
As the analysis develops, you can add new sections or details to this section.

## Notebooks should be runnable
Insofar as it's possible, another PUDL user who has cloned the repository that the notebook is part of should be able to update their `pudl-dev` conda environment, open the notebook, and run all cells successfully.
If there are required data or other prerequisites that the notebook cannot manage on its own -- like a file that needs to be downloaded by hand and placed in a particular location -- those steps should be laid out clearly at the beginning of the notebook.

## Avoid Troublesome Elements

### Don't hardcode passwords or access tokens
Most of our work is done in public Github repositories.
No authentication information should ever appear in a notebook.
These values can be stored in environment variables on your local computer.

### Do not hardocde values, especially late in the notebook
If the analysis depends on particular choices of input values, those should be called out explicitly at the beginning of the notebook.
(NB: We should explore ways to parameterize notebooks, [papermill](https://papermill.readthedocs.io/en/latest/) is one tool that does this).

### Don't hardcode absolute file paths
If anyone is going to be able to use the notebook, the files it uses will need to be stored somewhere that makes sense on both your and other computers.
We assume that anyone using this template has the PUDL package installed, and has a local PUDL data management environment set up.
  * Input data that needs to be stored on disk and accessed via a shared location should be saved under `<PUDL_IN>/data/local/<data_source>/`.
  * Intermediate saved data products (e.g. a pickled result of a computationally intensive process) and results should be saved to a location relative to the notebook, and within the directory hierarchy of the repository that the notebook is part of.
  
### Don't require avoidable long-running computations
Consider persisting to disk the results of computations that take more than a few minutes, if the outputs are small enough to be checked into the repository and shared with other users.
Only require the expensive computation to be run if this pre-computed output is not available.

### Don't litter
Don't leave lots of additional code laying around, even commented out, "just in case" you want to look at it again later.
Notebooks need to be relatively linear in the end, even though the thought processes and exploratory analyses that generate them may not be.
Once you have a working analysis, either prune those branches, or encapsulate them as options within functions.

### Don't load unneccesary libraries
Only import libraries which are required by the notebook, to avoid unnecessary dependencies.
If your analysis requires a new library that isn't yet part of the shared `pudl-dev` environment, add it to the `devtools/environment.yml` file so that others will get it when they update their environment.

## Related Resources:
Lots of these guidelines are taken directly from Emily Riederer's post: [RMarkdown Driven Development](https://emilyriederer.netlify.app/post/rmarkdown-driven-development/).
For more in depth explanation of the motivations behind this layout, do go check it out!

# Import Libraries
* Because it's very likely that you will be editing the PUDL Python packages or your own local module under development while working in the notebook, use the %autoreload magic with autoreload level 2 to ensure that any changes you've made in those files are always reflected in the code that's running in the notebook.
* Put all import statements at the top of the notebook, so everyone can see what its dependencies are up front, and so that if an import is going to fail, it fails early, before the rest of the notebook is run.
* Try to avoid importing individual functions and classes from deep within packages, as it may not be clear to other users where those elements came from, later in the notebook, and also to minimize the chance of namespace collisions.

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Standard libraries
import logging
import os
import pathlib
import sys

# 3rd party libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl
import pudl.constants as pc

# Configure Display Parameters

In [4]:
sns.set()
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,4)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Use Python Logging facilities
* Using a logger from the beginning will make the transition into the PUDL package easier.
* Creating a logging handler here will also allow you to see the logging output coming from PUDL and other underlying packages.

In [5]:
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

# Define Functions
In many cases, the eventual product of a notebook analysis is going to be the creation of new, reusable functions that are integrated into the underlying PUDL code. You should begin the process of accumulating and organizing those functions as soon as you notice repeated patterns in your code.
* Functions should be used to encapsulate any potentially reusable code.
* Functions should be defined immediately after the imports, to avoid accidental dependence on zombie variables that are defined further down in the code
* While they may evolve over time, having brief docstrings explaining what they do will help others understand them.
* If there's a particular type of plot or visualization that is made repeatedly in the notebook, it's good to parameterize and functionalize it here too, so that as you refine the presentation of the data and results, those improvements can be made in a single place, and shown uniformly throughout the notebook.
* As these functions mature and become more general purpose tools, you will probably want to start migrating them into their own local module, under a `src` directory in the same directory as the notebook. You will want to import this module 

## Dummy EIA 861 ETL

In [6]:
def test_etl_eia(eia_inputs, pudl_settings):
    """
    This is a dummy function that runs the first part of the EIA ETL
    process -- everything up until the entity harvesting begins. For
    use in this notebook only.

    """
    eia860_tables = eia_inputs["eia860_tables"]
    eia860_years = eia_inputs["eia860_years"]
    eia861_tables = eia_inputs["eia861_tables"]
    eia861_years = eia_inputs["eia861_years"]
    eia923_tables = eia_inputs["eia923_tables"]
    eia923_years = eia_inputs["eia923_years"]

    # generate CSVs for the static EIA tables, return the list of tables
    #static_tables = _load_static_tables_eia(datapkg_dir)
    
    # For new Datastore args
    pudl_in = pathlib.Path(pudl_settings['pudl_in'])
    ds = pudl.workspace.datastore.Datastore(pudl_in, sandbox=True)
    
    # Extract EIA forms 923, 860
    eia860_raw_dfs = pudl.extract.eia860.Extractor(ds).extract(eia860_years)
    eia861_raw_dfs = pudl.extract.eia861.Extractor(ds).extract(eia861_years)
    eia923_raw_dfs = pudl.extract.eia923.Extractor(ds).extract(eia923_years)

    # Transform EIA forms 860, 861, 923
    eia860_transformed_dfs = pudl.transform.eia860.transform(eia860_raw_dfs, eia860_tables=eia860_tables)
    eia861_transformed_dfs = pudl.transform.eia861.transform(eia861_raw_dfs, eia861_tables=eia861_tables)
    eia923_transformed_dfs = pudl.transform.eia923.transform(eia923_raw_dfs, eia923_tables=eia923_tables)

    # create an eia transformed dfs dictionary
    eia_transformed_dfs = eia860_transformed_dfs.copy()
    eia_transformed_dfs.update(eia861_transformed_dfs.copy())
    eia_transformed_dfs.update(eia923_transformed_dfs.copy())

    # convert types..
    eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(eia_transformed_dfs, 'eia')

    return eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs

# Define Notebook Parameters
If there are overarching parameters which determine the nature of the analysis -- which US states to look at, which utilities are of interest, a particular start and end date -- state those clearly at the beginning of the analysis, so that they can be referred to by the rest of the notebook and easily changed if need be.
* If the analysis depends on local (non-PUDL managed) datasets, define the paths to those data here.
* If there are external URLs or other resource locations that will be accessed, define those here as well.
* This is also where you should create your `pudl_settings` dictionary and connections to your local PUDL databases

In [7]:
EIA861_YEARS = list(range(2001, 2019))
pudl_settings = pudl.workspace.setup.get_defaults()
display(pudl_settings)

ferc1_engine = sa.create_engine(pudl_settings['ferc1_db'])
display(ferc1_engine)

pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
display(pudl_engine)

pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine)


# Is there other external data you need to pull in?
# If so, put it in a (relatively) standard place on the filesystem.
my_new_data_url = "https://mynewdata.website.gov/path/to/new/data.csv"
my_new_datadir = pathlib.Path(pudl_settings["data_dir"]) / "local/new_data_source"

# Store API keys and other secrets in environment variables
# and read them in here, if needed:
# API_KEY_EIA = os.environ["API_KEY_EIA "]
# API_KEY_FRED = os.environ["API_KEY_FRED "]

{'pudl_in': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR',
 'data_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/data',
 'settings_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/settings',
 'pudl_out': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR',
 'sqlite_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite',
 'parquet_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/parquet',
 'datapkg_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/datapkg',
 'notebook_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/notebook',
 'ferc1_db': 'sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/pudl.sqlite'}

Engine(sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/ferc1.sqlite)

Engine(sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/pudl.sqlite)

# Load Data
* Given the above parameters and functions, it should now be possible to load the required data into local variables for further wrangling, analysis, and visualization.
* If the data is not yet present on the machine of the person running the notebook, this step should also acquire the data from its original source, and place it in the appropriate location under `<PUDL_IN>/data/local/`.
* If there are steps which have to be done manually here, put them first so that they fail first if the user hasn't read the instructions, and they can fix the situation before a bunch of other work gets done. Try to minimize the amount of work in the filesystem that has to be done manually though.
* If this process takes a little while, don't be shy about producing `logging` output.
* Using the `%%time` cell magic can also help users understand which pieces of work / data acquisition are hard:

## EIA 861 (2010-2018)
* Not yet fully integrated into PUDL
* Post-transform harvesting process isn't compatible w/ EIA 861 structure
* Only getting the `sales_eia861`, `balancing_authority_eia861`, and `service_territory_eia861` tables

### Already Transformed EIA 861 DataFrames

In [8]:
%%time
eia_inputs = {
    "eia860_years": [],
    "eia860_tables": pudl.constants.pudl_tables["eia860"],
    "eia861_years": EIA861_YEARS,
    "eia861_tables": pudl.constants.pudl_tables["eia861"],
    "eia923_years": [],
    "eia923_tables": pudl.constants.pudl_tables["eia923"],
}
eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs = test_etl_eia(eia_inputs=eia_inputs, pudl_settings=pudl_settings)

No years given. Not extracting eia860 spreadsheet data.
Extracting eia861 spreadsheet data.


/Users/aesharpe/Desktop/Work/Catalyst_Coop/pudl/src/pudl/extract/eia861.py:39: UserWarning: Integration of EIA 861 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
No years given. Not extracting eia923 spreadsheet data.
No raw EIA 860 dataframes found. Not transforming EIA 860.
Transform

/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/missing.py:47: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


No raw EIA 923 DataFrames found. Not transforming EIA 923.
CPU times: user 2min 42s, sys: 5.7 s, total: 2min 47s
Wall time: 3min 8s


# Sanity Check Data
If there's any validation that can be done on the data which you've loaded to flag if/when it is inappropriate for the analysis that follows, do it here. If you find the data is unusable, use `assert` statements or `raise` Exceptions to stop the notebook from proceeding, and indicate what the problem is.

TO DO: 
    * compare totals col - result: total_capacity col is more accurate but need total_capacity_mw for conversion of pct to       mw
    * ensure that col breakdown is acurate:
        * that all columns are accounted for
        * that the "tech" cols are actually components that sum to the total
        * check on the wierd extra cols that are the "half" components.
    * deal with / record duplicates

In [17]:
list(eia_transformed_dfs.keys())

['service_territory_eia861',
 'balancing_authority_eia861',
 'sales_eia861',
 'advanced_metering_infrastructure_eia861',
 'demand_response_eia861',
 'demand_side_management_sales_eia861',
 'demand_side_management_program_customers_eia861',
 'demand_side_management_ee_dr_eia861',
 'demand_side_management_misc_eia861',
 'distributed_generation_tech_eia861',
 'distributed_generation_fuel_eia861',
 'distributed_generation_misc_eia861',
 'distribution_systems_eia861',
 'dynamic_pricing_eia861',
 'energy_efficiency_eia861',
 'green_pricing_eia861',
 'mergers_eia861',
 'net_metering_customer_fuel_class_eia861',
 'net_metering_misc_eia861',
 'non_net_metering_customer_fuel_class_eia861',
 'non_net_metering_misc_eia861',
 'operational_data_revenue_eia861',
 'operational_data_misc_eia861',
 'reliability_eia861',
 'utility_data_nerc_eia861',
 'utility_data_rto_eia861',
 'utility_data_misc_eia861',
 'balancing_authority_assn_eia861',
 'utility_assn_eia861']

# Preliminary Data Wrangling
Once all of the data is loaded and looks like it's in good shape, do any initial wrangling that's specific to this particular analysis. This should mostly make use of the higher level functions which were defined above. If this step takes a while, don't be shy about producing `logging` outputs.

DSM table TO-DO 
* add _make_yn_bool() and _thousand_to_one() to other tables [ ]
* backfill states? [ ]
* create check to see that all columns have a dtype in constants

In [118]:
eia_transformed_dfs['demand_side_management_program_customers_eia861'].columns.tolist()

['utility_id_eia',
 'state',
 'nerc_region',
 'report_date',
 'customer_class',
 'price_responsiveness_customers',
 'time_responsiveness_customers']

In [40]:
idx_cols = [
    'utility_id_eia',
    'state',
    'nerc_region',
    'report_date',
    #'entity_type' # ? 
]

sales_cols = [
    'sales_for_resale',
    'sales_to_ultimate_consumers'
]

bool_cols = [
    'energy_savings_estimates_independently_verified',
    'energy_savings_independently_verified',
    'major_program_changes',
    'price_responsive_programs',
    'short_form',
    'time_responsive_programs',
]

cost_cols = [
    'annual_indirect_program_cost',
    'annual_total_cost',
    'energy_efficiency_annual_cost',
    'load_management_annual_cost',
]

In [259]:
raw_dsm = eia_transformed_dfs['demand_side_management_eia861'].copy()

In [260]:
###########################################################################
# Transform Data Round 1 (must be done to avoid issues with nerc_region col in _tidy_class_dfs())
# * Clean NERC region col
# * Drop data_status and demand_side_management (all F) cols
###########################################################################

transformed_dsm1 = (
    pudl.transform.eia861._clean_nerc(raw_dsm, idx_cols)
    .drop(['demand_side_management', 'data_status'], axis=1)
    .query("utility_id_eia not in [88888]")
)

The following reported NERC regions are not currently recognized and become UNK values: []


In [261]:
# Separate dsm data into sales vs. other table (the latter of which can be tidied)
dsm_sales = transformed_dsm1[idx_cols + sales_cols].copy()
dsm_ee_dr = transformed_dsm1.drop(sales_cols, axis=1)

In [263]:
###########################################################################
# Tidy Data:
###########################################################################

tidy_dsm, dsm_idx_cols = (
    pudl.transform.eia861._tidy_class_dfs(
        dsm_ee_dr,
        df_name='Demand Side Management',
        idx_cols=idx_cols,
        class_list=pc.CUSTOMER_CLASSES,
        class_type='customer_class',
        keep_totals=True
    )
)

In [275]:
###########################################################################
# Transform Data Round 2
# * Make booleans (Y=True, N=False)
# * Turn 1000s of dollars back into dollars
###########################################################################

def _make_yn_bool(df_object):
    """Turn Y/N reporting into True or False boolean statements for df or series."""
    return df_object.replace({"N": False, "Y": True})

def _thousand_to_one(df_object):
    """Turn reporting in thousands of dollars to regular dollars for df or series."""
    return df_object * 1000

In [276]:
#Split tidy dsm data into transformable chunks
tidy_dsm_bool = tidy_dsm[dsm_idx_cols + bool_cols].copy().set_index(dsm_idx_cols)
tidy_dsm_cost = tidy_dsm[dsm_idx_cols + cost_cols].copy().set_index(dsm_idx_cols)
tidy_dsm_ee_dr = tidy_dsm.drop(bool_cols, axis=1).drop(cost_cols, axis=1)

# Calculate transformations for each chunk
transformed_dsm2_bool = (
    _make_yn_bool(tidy_dsm_bool)
    .reset_index()
    .assign(short_form=lambda x: x.short_form.fillna(False))
)
transformed_dsm2_cost = _thousand_to_one(tidy_dsm_cost).reset_index()

In [277]:
# Merge chunks back together
transformed_dsm2 = (
    pd.merge(transformed_dsm2_bool, transformed_dsm2_cost, on=dsm_idx_cols, how='outer')
)
transformed_dsm2 = (
    pd.merge(transformed_dsm2, tidy_dsm_ee_dr, on=dsm_idx_cols, how='outer')
)

In [278]:
# Split into final tables
ee_cols = [col for col in transformed_dsm2 if 'energy_efficiency' in col]
dr_cols = [col for col in transformed_dsm2 if 'load_management' in col]
program_cols = ['price_responsiveness_customers', 'time_responsiveness_customers']
total_cost_cols = ['annual_indirect_program_cost','annual_total_cost']

dsm_program_customers = transformed_dsm2[dsm_idx_cols + program_cols].copy()
dsm_ee_dr = transformed_dsm2[dsm_idx_cols + ee_cols + dr_cols + total_cost_cols]
dsm_misc = (
    transformed_dsm2.drop(
        ee_cols 
        + dr_cols 
        + program_cols 
        + total_cost_cols 
        + ['customer_class'], axis=1)
    .drop_duplicates()
)

In [279]:
# Final Tables
dsm_program_customers
dsm_ee_dr
dsm_misc
dsm_sales

,utility_id_eia,state,nerc_region,report_date,sales_for_resale,sales_to_ultimate_consumers
0,36189,<NA>,ECAR,2001-01-01,NaN,131472.0
1,38084,<NA>,MAPP,2001-01-01,NaN,1028712.0
2,30518,<NA>,WSCC,2001-01-01,NaN,69970.0
3,20396,<NA>,MAPP,2001-01-01,NaN,10387.0
4,20404,<NA>,ERCOT,2001-01-01,3535043.0,4752432.0
...,...,...,...,...,...,...
11937,57483,CA,UNK,2012-01-01,NaN,NaN
11938,58124,FL,UNK,2012-01-01,NaN,NaN
11939,58125,CA,UNK,2012-01-01,NaN,NaN
11940,58127,MA,UNK,2012-01-01,NaN,NaN


## Fuzzy Match Columns

In [85]:
from fuzzywuzzy import fuzz

def fuzzy_match(df, col1, col2):
    # Define base comparison df 
    eia_base_df = (
        pudl_out.utils_eia860()
        #[['report_date', 'utility_id_eia', 'utility_id_pudl', 'utility_name_eia', 'state']]
        .assign(utility_id_eia=lambda x: x.utility_id_eia.astype('Int64'))
    )
    # Shorten df
    #df = df[['utility_id_eia', 'state', 'utility_name_eia', 'report_date']]
    
    # Merge dfs together 
    df_merge = (
        pd.merge(eia_base_df, df, on=['utility_id_eia', 'state'], how='outer', suffixes=('_base', '_861'))
        .set_index(['utility_id_eia', 'state'])
        .assign(utility_name_eia_base=lambda x: x.utility_name_eia_base.astype('string'),
                utility_name_eia_861=lambda x: x.utility_name_eia_861.astype('string'))
    )
    
    # Only run where both columns for utility name have values
    df_merge_no_null = (
        df_merge.loc[
            (df_merge[col1].notnull()) 
            & (df_merge[col2].notnull())].copy()
    )
    
    # Fuzzy match
    df_merge_no_null['fuzzy'] = (
        df_merge_no_null.apply(lambda x: fuzz.ratio(x.utility_name_eia_base, x.new_parent),axis=1)
    )
#     df_merge_no_null = (
#         df_merge_no_null.assign(fuzzy=lambda x: fuzz.ratio(x[col1], x[col2])))
    
    #df_merge_no_null.loc[:,'fuzzy'] = fuzzy_series
    df_merge_no_null = df_merge_no_null.sort_values('fuzzy')
    
    return df_merge_no_null

# DSM Stuff

In [37]:
dsm = eia861_raw_dfs['demand_side_management_eia861'].copy()
#dsm.columns.tolist()

In [50]:
idx_cols = ['utility_id_eia', 'state', 'ba_code', 'report_year']

dsm_test = dsm[['utility_id_eia', 'state', 'ba_code', 'report_year', 
                'residential_annual_load_management_energy_effects ', 
                'commercial_annual_load_management_energy_effects ', 
                'industrial_annual_load_management_energy_effects ',
                'transportation_annual_load_management_energy_effects ',
                'total_annual_load_management_energy_effects ',
]]

dr_test = raw_dr_eia861[['utility_id_eia', 'state', 'balancing_authority_code_eia', 'report_year',
                         'residential_energy_savings_mwh',
                         'commercial_energy_savings_mwh',
                         'industrial_energy_savings_mwh',
                         'transportation_energy_savings_mwh',
                         'total_energy_savings_mwh'
]].rename(columns={'balancing_authority_code_eia': 'ba_code'})

In [84]:
merge = pd.merge(dsm_test, dr_test, on=idx_cols, how='outer')
merge = pudl.helpers.oob_to_nan(merge, 
                       ['residential_annual_load_management_energy_effects ', 
                        'commercial_annual_load_management_energy_effects ', 
                        'industrial_annual_load_management_energy_effects ',
                        'transportation_annual_load_management_energy_effects ',
                        'residential_energy_savings_mwh',
                        'commercial_energy_savings_mwh',
                        'industrial_energy_savings_mwh',
                        'transportation_energy_savings_mwh',
                        'total_energy_savings_mwh',
                        'total_annual_load_management_energy_effects '])

In [71]:
dr_col = 'residential_energy_savings_mwh'
dsm_col = 'residential_annual_load_management_energy_effects '

In [36]:
merge = merge.loc[merge['report_year'].isin(range(2012,2014))]
merge = merge.loc[merge[dr_col].notna()]
#merge = merge.loc[merge[dsm_col].notna()]


NameError: name 'merge' is not defined

In [34]:
#test.plot.scatter('residential_annual_load_management_energy_effects ', 'residential_energy_savings_mwh')

In [33]:
#raw_dr_eia861.columns.tolist()

# Data Analysis and Visualization
* Now that you've got the required data in a usable form, you can tell the story of your analysis through a mix of visualizations, and further data wrangling steps.
* This narrative should be readable, with figures that have titles, legends, and labeled axes as appropriate so others can understand what you're showing them.
* The code should be concise and make use of the parameters and functions which you've defined above when possible. Functions should contain comprehensible chunks of work that make sense as one step in the story of the analysis.